In [2]:
import unittest
from blinker import signal, Signal
import tempfile
import silq
from silq.tools.config import *
silq.initialize('Serwan')

Initializing imports
Initialization complete


In [3]:
%%javascript

Jupyter.keyboard_manager.command_shortcuts.add_shortcut('r', {
    help : 'run cell',
    help_index : 'zz',
    handler : function (event) {
        IPython.notebook.restart_run_all();
        return false;
    }}
);

<IPython.core.display.Javascript object>

In [4]:
class TestPulseSignals(unittest.TestCase):
    def setUp(self):
        self.signal = signal('pulse')
        self.is_set = False 
        self.dict = {}

    def test_set_val(self):
        self.assertFalse(self.is_set)
        self.set_val(self)
        self.assertTrue(self.is_set)

    def test_signal_emit(self):
        p = Pulse()
        self.signal.connect(self.set_dict)
        self.assertFalse('t_start' in self.dict)
        p.t_start = 1
        self.assertEqual(1, self.dict['t_start'])

    def test_subsequent_pulses(self):
        p1 = Pulse(t_start=0, t_stop=10)
        self.assertEqual(p1.t_start, 0)
        self.assertEqual(p1.duration, 10)
        self.assertEqual(p1.t_stop, 10)
        
        p2 = Pulse(t_start=PulseMatch(p1, 't_stop', delay=1), duration=4)
        self.assertEqual(p2.t_start, 11)
        self.assertEqual(p2.duration, 4)
        self.assertEqual(p2.t_stop, 15)
        
        p1.t_stop = 14
        self.assertEqual(p1.t_start, 0)
        self.assertEqual(p1.duration, 14)
        self.assertEqual(p1.t_stop, 14)
        
        self.assertEqual(p2.t_start, 15)
        self.assertEqual(p2.duration, 4)
        self.assertEqual(p2.t_stop, 19)
        
        p2.t_start = 0
        self.assertEqual(p2.t_start, 0)
        self.assertEqual(p2.duration, 4)
        self.assertEqual(p2.t_stop, 4)
        
        p1.t_stop = 20
        self.assertEqual(p2.t_start, 0)
        self.assertEqual(p2.duration, 4)
        self.assertEqual(p2.t_stop, 4)

    def set_val(self, sender, **kwargs):
        self.is_set = True

    def set_dict(self, sender, **kwargs):
        self.dict.update(**kwargs)

In [5]:
class TestPulseConfig(unittest.TestCase):
    def setUp(self):
        self.signal = signal('config:pulses.read')
        
        pulses_dict = {'read': {}}
        self.pulses_config = DictConfig(name='pulses',
                                        folder=None,
                                        config=pulses_dict)
        qc.config.user.pulses = self.pulses_config
        self.pulse_config = self.pulses_config.read
        self.dict = {}
        Pulse.pulses_config = self.pulses_config

    def tearDown(self):
        for key in self.pulses_config:
            signal('config:pulses.'+key).receivers = {}

    def test_set_item(self):
        with self.assertRaises(KeyError):
            self.pulse_config['duration']
        self.pulse_config['duration'] = 1
        self.assertEqual(self.pulse_config['duration'], 1)
        self.assertEqual(self.pulse_config.duration, 1)

    def test_signal(self):
        self.signal.connect(self.set_dict)
        self.pulse_config.duration = 1
        self.assertIn('duration', self.pulse_config)
        self.assertEqual(self.pulse_config.duration, 1)
        
        self.pulses_config.read2 = {'t_start': 'config:pulses.read.duration'}
        self.assertIsInstance(self.pulses_config.read2, DictConfig)
        
        self.assertEqual(self.pulses_config.read2.t_start, 1)
        
        signal('config:pulses.read').connect(self.set_dict)
        signal('config:pulses.read2').connect(self.set_dict)
        
        self.pulse_config.duration = 3
        self.assertEqual(self.dict['duration'], 3)
        self.assertEqual(self.dict['t_start'], 3)
        
        self.pulses_config.read2.t_start = 20
        self.dict = {}
        self.pulse_config.duration = 5
        self.assertEqual(self.dict['duration'], 5)
        self.assertNotIn('t_start', self.dict)

    def test_config(self):
        self.assertIsInstance(self.pulse_config, DictConfig)
        self.pulse_config.t_start = 4
        self.assertEqual(self.pulse_config.t_start, 4)
        self.assertEqual(self.pulses_config.read.t_start, 4)
        
        # Create new dict
        d = {'read2': {'t_start': 1, 'subdict': {'t_start': 2}}}
        dict_config = DictConfig(name='iterdict', config=d)
        self.assertIsInstance(dict_config.read2, DictConfig)
        self.assertIsInstance(dict_config.read2.subdict, DictConfig)
        self.assertEqual(dict_config.read2.t_start, 1)
        self.assertEqual(dict_config.read2.subdict.t_start, 2)

    def test_pulse_from_config(self):
        self.pulse_config.t_start = 10
        p = Pulse(name='read')
        self.assertEqual(p.t_start, 10)
        
        p.t_start = 20
        self.assertEqual(p.t_start, 20)
        
        self.pulse_config.t_start = 0
        self.assertEqual(p.t_start, 0)
        
        self.pulse_config.t_start = 'config:pulses.read.t_stop'
        with self.assertRaises(AttributeError):
            self.pulse_config.t_start
            
        self.pulse_config.t_stop = 50
        self.assertEqual(self.pulse_config.t_start, 50)
        self.assertEqual(p.t_start, 50)
        
        self.pulse_config.t_start = 40
        self.assertEqual(self.pulse_config.t_start, 40)
        self.assertEqual(p.t_start, 40)
        
        self.pulse_config.t_stop = 60
        self.assertEqual(self.pulse_config.t_start, 40)
        self.assertEqual(p.t_start, 40)

    def set_dict(self, sender, **kwargs):
        self.dict.update(**kwargs)

In [6]:
d = tempfile.TemporaryDirectory()

In [7]:
d.cleanup

<bound method TemporaryDirectory.cleanup of <TemporaryDirectory 'C:\\Users\\Serwan\\AppData\\Local\\Temp\\tmpz0q8pwgm'>>

In [8]:
class TestConfig(unittest.TestCase):
    def setUp(self):
        self.folder = tempfile.TemporaryDirectory()
        self.d = {
            'pulses': {
                'read': {'t_start': 0,
                         't_stop': 10}},
            'connections': ['connection1', 'connection2']}
        self.config = DictConfig('env1', folder=self.folder.name,
                                 config=self.d)
        
    def tearDown(self):
        self.folder.cleanup()
        
    def test_dicts_equal(self):
        self.assertTrue(self.dicts_equal(self.d, self.d))
        self.assertTrue(self.dicts_equal(self.d, self.config))
        self.assertTrue(self.dicts_equal(self.config, self.config))
        self.config.x = 2
        self.assertFalse(self.dicts_equal(self.d, self.config))
        
    def test_save_load_dir(self):
        self.config.save(folder=self.folder.name, save_as_dir=True)
        
        filepath = os.path.join(self.folder.name, '{}.json'.format(self.config.name))
        folderpath = os.path.join(self.folder.name, self.config.name)
        self.assertFalse(os.path.exists(filepath))
        self.assertTrue(os.path.isdir(folderpath))
        for filename in self.config.keys():
            filepath = os.path.join(folderpath, '{}.json'.format(filename))
            self.assertTrue(os.path.exists(filepath))
        
        config_loaded = DictConfig('env1', folder=self.folder.name)
        self.assertFalse(config_loaded.save_as_dir)
        self.assertTrue(self.dicts_equal(self.config, config_loaded))
        
        config2 = DictConfig('env1', folder=self.folder.name)
        self.assertTrue(config2.save_as_dir)
        self.assertTrue(self.dicts_equal(self.config, self.config2))
        
    def test_save_load_file(self):
        self.config.save(folder=self.folder.name)
        
        filepath = os.path.join(self.folder.name, '{}.json'.format(self.config.name))
        folderpath = os.path.join(self.folder.name, self.config.name)
        self.assertTrue(os.path.exists(filepath))
        self.assertFalse(os.path.exists(folderpath))
        
        config_loaded = DictConfig('env1', folder=self.folder.name)
        self.assertFalse(config_loaded.save_as_dir)
        self.assertTrue(self.dicts_equal(self.config, config_loaded))
        
        
    def dicts_equal(self, d1, d2):
        d1_keys = list(d1.keys())
        for d2_key, d2_val in d2.items():
            if d2_key not in d1_keys:
                return False
            else:
                d1_keys.remove(d2_key)
                d1_val = d1[d2_key]
                if isinstance(d1_val, dict):
                    if not isinstance(d2_val, dict):
                        return False
                    if not self.dicts_equal(d1_val, d2_val):
                        return False
                elif not d1_val == d2_val:
                    return False
        return True

In [9]:
# Run tests    
suite_pulse_signals = unittest.TestLoader().loadTestsFromModule(TestPulseSignals())
suite_pulse_config = unittest.TestLoader().loadTestsFromModule(TestPulseConfig())
suite_config = unittest.TestLoader().loadTestsFromModule(TestConfig())
suite = unittest.TestSuite([suite_pulse_signals, suite_pulse_config, suite_config])
unittest.TextTestRunner(verbosity=2, failfast=True).run(suite)

test_set_val (__main__.TestPulseSignals) ... ok
test_signal_emit (__main__.TestPulseSignals) ... ok
test_subsequent_pulses (__main__.TestPulseSignals) ... ok
test_config (__main__.TestPulseConfig) ... ok
test_pulse_from_config (__main__.TestPulseConfig) ... ok
test_set_item (__main__.TestPulseConfig) ... ok
test_signal (__main__.TestPulseConfig) ... ok
test_dicts_equal (__main__.TestConfig) ... ok
test_save_load_dir (__main__.TestConfig) ... FAIL

FAIL: test_save_load_dir (__main__.TestConfig)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-8-ac3e6c37cf68>", line 38, in test_save_load_dir
    self.assertTrue(config2.save_as_dir)
AssertionError: False is not true

----------------------------------------------------------------------
Ran 9 tests in 0.030s

FAILED (failures=1)


<unittest.runner.TextTestResult run=9 errors=0 failures=1>